In [1]:
import findspark
findspark.init('/home/aarav/Amit/SJSU/CMPE_272/mini_project/spark_setup/spark-2.4.5-bin-hadoop2.6')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
data = spark.read.csv('../dataset/train_mosaic.csv', header=True, inferSchema=True)

In [4]:
data.show()

+----------------+-------------+-----------------+----------------------+---------------------------+---------------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+-----------------+-------------------+-------------+------------------+------------+------------+-------------+------------------+-----------------+-----------+-----------+-------------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+-----------------+-----------------+---------------+--------------------+-----------------+-----------------+------------------+-----------------+----------------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+------------------

In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Label", outputCol="LabelIndex")
indexed = indexer.fit(data).transform(data)
new_data = indexed.drop("Label")
new_data.show()

+----------------+-------------+-----------------+----------------------+---------------------------+---------------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+-----------------+-------------------+-------------+------------------+------------+------------+-------------+------------------+-----------------+-----------+-----------+-------------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+-----------------+-----------------+---------------+--------------------+-----------------+-----------------+------------------+-----------------+----------------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+------------------

In [7]:
#feature_columns = new_data.columns['Destination_Port','Flow_Duration','Total_Fwd_Packets','Total_Backward_Packets','Total_Length_of_Fwd_Packets','Total_Length_of_Bwd_Packets'] # here we omit the final 2 columns
#feature_columns = ['Destination_Port','Flow_Duration','Total_Fwd_Packets','Total_Backward_Packets','Total_Length_of_Fwd_Packets','Total_Length_of_Bwd_Packets']
feature_columns = data.columns[:-2]
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [8]:
data_2 = assembler.transform(new_data)

In [155]:
#data_2.select("features").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                               

In [9]:
data_2.show()

+----------------+-------------+-----------------+----------------------+---------------------------+---------------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+---------------------+-----------------+-------------------+-------------+------------------+------------+------------+-------------+------------------+-----------------+-----------+-----------+-------------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+-----------------+-----------------+---------------+--------------------+-----------------+-----------------+------------------+-----------------+----------------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+------------------

In [10]:
train, test = data_2.randomSplit([0.7, 0.3])

In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
algo = LinearRegression(featuresCol="features", labelCol="LabelIndex")

In [13]:
model = algo.fit(train)

In [14]:
evaluation_summary = model.evaluate(test)

In [15]:
evaluation_summary.meanAbsoluteError

0.11703062724766732

In [16]:
evaluation_summary.rootMeanSquaredError

0.19318063123871895

In [17]:
evaluation_summary.r2

0.9293118877047606

In [18]:
predictions = model.transform(test)

In [96]:
predictions.select(predictions.columns[75:]).limit(10).toPandas()

,Idle_Max,Idle_Min,LabelIndex,features,prediction
0,0.0,0.0,0.0,"(0.0, 48.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.043924
1,0.0,0.0,0.0,"(0.0, 7622.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",-0.049704
2,0.0,0.0,0.0,"(0.0, 21391.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",-0.013264
3,0.0,0.0,0.0,"(0.0, 47110.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",-0.055061
4,12423731.0,12423731.0,0.0,"(0.0, 12429379.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.079529
5,13504626.0,6103114.0,0.0,"(0.0, 112674470.0, 127.0, 0.0, 0.0, 0.0, 0.0, ...",0.440870
6,8693884.0,5019632.0,0.0,"(0.0, 116228634.0, 243.0, 0.0, 0.0, 0.0, 0.0, ...",0.353725
7,12001049.0,5488142.0,0.0,"(0.0, 116657024.0, 193.0, 0.0, 0.0, 0.0, 0.0, ...",0.583986
8,17854551.0,5489425.0,0.0,"(0.0, 117375861.0, 195.0, 0.0, 0.0, 0.0, 0.0, ...",0.399753
9,18384156.0,6509241.0,0.0,"(0.0, 117595795.0, 172.0, 0.0, 0.0, 0.0, 0.0, ...",0.425107
